**Setup Ollama**

In [1]:
# !curl -fsSL https://ollama.com/install.sh | sh
# !nohup ollama serve &
# !ollama pull qwen3:4b
# !pip install -q mcp-server sentence-transformers  PyMuPDF faiss-cpu
# !pip install -q gradio
# !pip install -q langchain-community
# !pip install -U --quiet langgraph
# !pip install -q python-docx
# !pip install -q chromadb

In [2]:
# !pip install pdfplumber

In [3]:
from __future__ import annotations

import os
import re
import uuid
import json
import queue
import threading
import tempfile
from pathlib import Path
from typing import List, Dict, Tuple, Any

import gradio as gr
import pdfplumber
import docx
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
import requests

d:\anaconda3\envs\mcp_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Count page number

In [4]:
from pypdf import PdfReader

# Replace 'yourfile.pdf' with your actual file name, e.g. '/mnt/data/yourfile.pdf'
pdf_path = 'D:/Download/Facture.pdf'

# Open the PDF file
with open(pdf_path, 'rb') as file:
    reader = PdfReader(file)
    num_pages = len(reader.pages)
    print(f"Total pages: {num_pages}")


Total pages: 8


### count paragraph

In [5]:
import pdfplumber
import fitz  # PyMuPDF
import re
def extract_text_pdfplumber(pdf_path):
    all_text = ""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                page_text = page.extract_text()
                if page_text:
                    page_text = page_text.replace('-\n', '')  # Fix hyphenated line breaks
                    all_text += page_text + "\n"
    except Exception as e:
        print("pdfplumber extraction failed:", e)
    return all_text

def extract_text_pymupdf(pdf_path):
    all_text = ""
    try:
        with fitz.open(pdf_path) as pdf:
            for page in pdf:
                text = page.get_text()
                all_text += text + "\n"
    except Exception as e:
        print("PyMuPDF extraction failed:", e)
    return all_text

def count_paragraphs(pdf_path):
    # Try with pdfplumber first
    all_text = extract_text_pdfplumber(pdf_path)
    if not all_text.strip():
        # Fallback to PyMuPDF if nothing was extracted
        print("Trying PyMuPDF as fallback...")
        all_text = extract_text_pymupdf(pdf_path)
        if not all_text.strip():
            print("Text extraction failed.")
            return 0

    # Split into paragraphs (two or more consecutive newlines)
    paragraphs = [p for p in re.split(r'\n{2,}', all_text) if p.strip()]
    
    # Show first 5 paragraphs for quality check
    print("\n--- Sample Paragraphs ---")
    for i, para in enumerate(paragraphs[:5], 1):
        print(f"\nParagraph {i}:\n{para}\n")
    
    return len(paragraphs)

# Example usage
pdf_path = 'D:/Download/LoRA.pdf'
print("Total paragraphs:", count_paragraphs(pdf_path))


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox



--- Sample Paragraphs ---

Paragraph 1:
Analyzing the Impact of Low-Rank Adaptation for
Cross-Domain Few-Shot Object Detection in Aerial
Images
Hicham TALAOUBRID Anissa MOKRAOUI Ismail BEN AYED
L2TI & COSE L2TI LIVIA, ETS
Université Sorbonne Paris Nord Université Sorbonne Paris Nord Montreal, Canada
hicham.talaoubrid1@edu.univ-paris13.fr anissa.mokraoui@univ-paris13.fr Ismail.BenAyed@etsmtl.ca
Axel PROUVOST Sonimith HANG Monit KORN Rémi HARVEY
IMT Mines Alès, France IMT Mines Alès, France IMT Mines Alès, France COSE, Montmagny, France
axel.prouvost@etu.mines-ales.fr sonimith.hang@etu.mines-ales.fr monit.korn@etu.mines-ales.fr remi.harvey@cose.fr
Abstract—This paper investigates the application of Low- domains, the problem becomes even more complex due
Rank Adaptation (LoRA) to small models for cross-domain to significant distribution shifts between source and target
few-shot object detection in aerial images. Originally designed
domains. Traditional fine-tuning approaches often strugg

In [6]:
# from langchain_community.embeddings import HuggingFaceEmbeddings
# from langchain_community.vectorstores import Chroma
# from langchain_core.documents import Document

# # Use a BGE embedding model from HuggingFace
# bge_embedding = HuggingFaceEmbeddings(
#     model_name="BAAI/bge-m3",
#     encode_kwargs={"normalize_embeddings": True}
# )

# vectorstore = Chroma.from_documents(doc_splits, bge_embedding, persist_directory="./chroma_db")
# retriever = vectorstore.as_retriever()

In [7]:
# from langchain.tools.retriever import create_retriever_tool

# retriever_tool = create_retriever_tool(
#     retriever,
#     name="search_user_documents",
#     description="Searches the user's uploaded documents and returns the most relevant passages with page and paragraph and document info.",
# )

In [8]:
# retriever_tool.invoke({"query": "what is LIME ?"})

In [9]:
# from langchain_community.llms.ollama import Ollama
# from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# # model = Ollama(model="Qwen/Qwen1.5-14B-Chat-GPTQ-Int4")
# # MODEL_NAME = "Qwen/Qwen1.5-14B-Chat-GPTQ-Int4"

# # tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# # model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, device_map="auto", trust_remote_code=True)

# # pipe = pipeline(
# #     "text-generation",
# #     model=model,
# #     tokenizer=tokenizer,
# #     max_new_tokens=1024,
# #     temperature=0,
# #     do_sample=True,
# # )

# # response_model = Ollama(model="Qwen/Qwen1.5-14B-Chat-GPTQ-Int4")
# response_model = Ollama(model="qwen3:4b")

# def generate_query_or_respond(state):
#     user_message = state["messages"][-1]["content"]
#     # 1. Run retriever/tool for context
#     retrieved_docs = retriever_tool.run(user_message)  # Or retriever.get_relevant_documents(...)
#     context = "\n\n".join([
#         doc.page_content if hasattr(doc, "page_content") else str(doc)
#         for doc in retrieved_docs
#     ])
#     # 2. Compose prompt for LLM
#     prompt = f"Use only the following context to answer:\n{context}\n\nUser question: {user_message}\nAnswer:"
#     # 3. Get LLM response
#     answer = response_model.invoke(prompt)
#     # 4. Return as a new message
#     return {"messages": state["messages"] + [{"role": "assistant", "content": answer}]}


In [10]:
# input = {
#     "messages": [
#         {
#             "role": "user",
#             "content": "What does the paper talk about?",
#         }
#     ]
# }
# output = generate_query_or_respond(input)


In [11]:
# print(output["messages"][-1]["content"])

In [12]:
# import gradio as gr
# import tempfile
# from pathlib import Path
# import pdfplumber

# from langchain.docstore.document import Document
# from langchain_text_splitters import RecursiveCharacterTextSplitter
# from langchain_community.embeddings import HuggingFaceEmbeddings
# from langchain_community.vectorstores import Chroma
# from langchain.tools.retriever import create_retriever_tool
# from langchain_community.llms.ollama import Ollama

# embedding_model = HuggingFaceEmbeddings(
#     model_name="BAAI/bge-m3",
#     encode_kwargs={"normalize_embeddings": True}
# )
# response_model = Ollama(model="qwen3:4b")

# def pdf_qa_pipeline(pdf_file, query_text):
#     # 1. Save file
#     temp_path = Path(tempfile.gettempdir()) / pdf_file.name
#     temp_path.write_bytes(pdf_file.read())
#     # 2. Extract text
#     text = ""
#     with pdfplumber.open(str(temp_path)) as pdf:
#         for page in pdf.pages:
#             page_text = page.extract_text()
#             if page_text:
#                 text += page_text
#     # 3. Wrap in Document
#     doc_obj = Document(page_content=text, metadata={"source": pdf_file.name})
#     docs_list = [doc_obj]
#     # 4. Split into chunks
#     splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
#         chunk_size=300, chunk_overlap=50
#     )
#     doc_splits = splitter.split_documents(docs_list)
#     # 5. Embed and build vectorstore
#     vectordb = Chroma.from_documents(doc_splits, embedding_model)
#     retriever = vectordb.as_retriever()
#     retriever_tool = create_retriever_tool(
#         retriever,
#         name="search_pdf",
#         description="Search the uploaded PDF for relevant information."
#     )
#     # 6. Retrieve relevant context
#     retrieved_docs = retriever_tool.run(query_text)
#     context = "\n\n".join([
#         doc.page_content if hasattr(doc, "page_content") else str(doc)
#         for doc in retrieved_docs
#     ])
#     # 7. Prompt LLM and return answer
#     prompt = f"Use only the following context to answer:\n{context}\n\nUser question: {query_text}\nAnswer:"
#     answer = response_model.invoke(prompt)
#     return answer

# demo = gr.Interface(
#     fn=pdf_qa_pipeline,
#     inputs=[
#         gr.File(label="Upload PDF", file_types=[".pdf"]),
#         gr.Textbox(label="Question", placeholder="Ask anything about the PDF..."),
#     ],
#     outputs=gr.Textbox(label="Answer"),
#     title="PDF Q&A - All In One",
#     description="Upload a PDF and ask a question—get an LLM answer grounded only in your document."
# )

# if __name__ == "__main__":
#     demo.launch(mcp_server=True)


**Handle file**

In [13]:
from pathlib import Path

def get_file_bytes_and_name(pdf_file):
    print("DEBUG: pdf_file type:", type(pdf_file))
    print("DEBUG: pdf_file dir:", dir(pdf_file))
    print("DEBUG: pdf_file repr:", repr(pdf_file))
    # Standard file-like object (e.g. Python's open, script mode)
    if hasattr(pdf_file, "read"):
         return pdf_file.read(), Path(pdf_file.name).name
    if isinstance(pdf_file, str):
        file_path = Path(pdf_file)
        with open(file_path, "rb") as f:
            return f.read(), file_path.name
    # else:
    #     raise ValueError("Could not extract file bytes from uploaded file.")
    raise ValueError("Could not extract file bytes from uploaded file.")


**Create Vector database**

In [14]:
# import tempfile
# from pathlib import Path
# import pdfplumber
# from langchain.docstore.document import Document
# from langchain_text_splitters import RecursiveCharacterTextSplitter
# from langchain_community.vectorstores import Chroma
# import re

# def build_vectorstore_from_pdf(pdf_file, embedding_model, persist_directory=None):


#     # --- read bytes & filename ---
#     file_bytes, file_name = get_file_bytes_and_name(pdf_file)

#     # --- write to a temp file ---
#     temp_path = Path(tempfile.gettempdir()) / file_name
#     with open(temp_path, "wb") as f:
#         f.write(file_bytes)

#     # Extract text and page number and paragraph
#     ##new code
#     # docs_list = []
#     ##end new code
#     text = ""
#     with pdfplumber.open(str(temp_path)) as pdf:
#       ##new code
#       # for page_num, page in enumerate(pdf.pages):
#       #   page_text = page.extract_text()
#       #   if page_text and page_text.strip():
#       #       doc_obj = Document(
#       #           page_content=page_text,
#       #           metadata={"source": file_name, "page": page_num + 1}
#       #       )
#       #       docs_list.append(doc_obj)
#         ##old code
#         for page in pdf.pages:
#             page_text = page.extract_text()
#             if page_text:
#                 text += page_text
#     doc_obj = Document(page_content=text, metadata={"source": pdf_file.name})
#     docs_list = [doc_obj]
#     # Split
#     splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
#         chunk_size=700, chunk_overlap=100
#     )
#     doc_splits = splitter.split_documents(docs_list)
#     # Build vectorstore
#     vectordb = Chroma.from_documents(
#         doc_splits,
#         embedding_model,
#         persist_directory=persist_directory,
#     )
#     return vectordb, page_count


### new version of the function

In [15]:
import tempfile
from pathlib import Path
import pdfplumber
from langchain.docstore.document import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
import re

def build_vectorstore_from_pdf(pdf_file, embedding_model, persist_directory=None):


    # --- read bytes & filename ---
    file_bytes, file_name = get_file_bytes_and_name(pdf_file)

    # --- write to a temp file ---
    temp_path = Path(tempfile.gettempdir()) / file_name
    with open(temp_path, "wb") as f:
        f.write(file_bytes)

    # Extract text and page number and paragraph
    ##new code
    # docs_list = []
    ##end new code
    text = ""
    docs_list = []
    page_count = 0
    with pdfplumber.open(str(temp_path)) as pdf:
        page_count = len(pdf.pages)
        for page_num, page in enumerate(pdf.pages, start=1):
            page_text = page.extract_text()
            if page_text:
                # Split into paragraphs (simple split by double newline)
                paragraphs = [p for p in page_text.split('\n\n') if p.strip()]
                for para_num, para in enumerate(paragraphs, start=1):
                    doc_obj = Document(
                        page_content=para,
                        metadata={
                            "source": file_name,
                            "page": page_num,
                            "paragraph": para_num,
                            "total_pages": page_count
                        }
                    )
                    docs_list.append(doc_obj)
    # Split
    splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=700, chunk_overlap=100
    )
    doc_splits = splitter.split_documents(docs_list)
    # Build vectorstore
    vectordb = Chroma.from_documents(
        doc_splits,
        embedding_model,
        persist_directory=persist_directory,
    )
    return vectordb, page_count


**Create tool**

In [16]:
def build_retriever_tool(vectorstore, name="search_user_documents"):
    retriever = vectorstore.as_retriever()
    from langchain.tools.retriever import create_retriever_tool
    retriever_tool = create_retriever_tool(
        retriever,
        name=name,
        description="Searches uploaded documents and returns relevant passages."
    )
    return retriever_tool


In [17]:
# !ollama pull qwen3:4b

In [18]:
# !nohup ollama serve &

In [19]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.llms.ollama import Ollama
import traceback
import torch

# Check if CUDA is available and set the device accordingly
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

embedding_model = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3",
    encode_kwargs={"normalize_embeddings": True, "device": device} # Specify device
)
response_model = Ollama(model="qwen3:4b")

# def pdf_qa_pipeline(pdf_file, query_text):
#     vectorstore = build_vectorstore_from_pdf(pdf_file, embedding_model)
#     retriever_tool = build_retriever_tool(vectorstore)
#     retrieved_docs = retriever_tool.run(query_text)
#     context = "\n\n".join([
#         doc.page_content if hasattr(doc, "page_content") else str(doc)
#         for doc in retrieved_docs
#     ])
#     prompt = f"Use only the following context to answer:\n{context}\n\nUser question: {query_text}\nAnswer:"
#     answer = response_model.invoke(prompt)
#     return answer
def pdf_qa_pipeline(pdf_file, query_text):
    # print("TYPE:", type(pdf_file))
    # print("DIR:", dir(pdf_file))
    # print("REPR:", repr(pdf_file))
    try:
        print("Starting pipeline...")
        vectorstore, page_count  = build_vectorstore_from_pdf(pdf_file, embedding_model)
        print("Vectorstore built.")
        retriever_tool = build_retriever_tool(vectorstore)
        print("Retriever tool built.")
        retrieved_docs = retriever_tool.run(query_text)
        print("Retrieved docs:", retrieved_docs)
        context_chunks = []
        for doc in retrieved_docs:
            if hasattr(doc, "metadata") and hasattr(doc, "page_content"):
                page = doc.metadata.get('page', '?')
                print("DEBUG: page:", page)
                para = doc.metadata.get('paragraph', '?')
                content = doc.page_content
            else:
                page = '?'
                para = '?'
                content = str(doc)
            context_chunks.append(f"[Page {page}, Paragraph {para}] {content}")
        context = "\n\n".join(context_chunks)  ##new code
        # context_chunks = []
        # for doc in retrieved_docs:
        #     if hasattr(doc, "metadata") and hasattr(doc, "page_content"):
        #         page = doc.metadata.get('page', '?')
        #         para = doc.metadata.get('paragraph', '?')
        #         content = doc.page_content
        #     else:
        #         page = '?'
        #         para = '?'
        #         content = str(doc)
        #     context_chunks.append(f"[Page {page}, Paragraph {para}] {content}")
        # context = "\n\n".join(context_chunks)
        # ---- 1. Print the context chunk you are sending to the LLM ----
        print("CONTEXT PASSED TO LLM:", context)

        # ---- 2. Return a default answer if the context is empty ----
        if not context.strip():
            return "No relevant information found in the uploaded document."

        # ---- 3. Use a stricter prompt ----
        prompt = f"""Answer the user's question **using only** the information provided by the file uploaded by ther user.
    Provide both the page and paragraph that you have found to support your response.
    If the context does not contain the answer, say 'Not found in the document.'

    Context:
    {context}

    Question: {query_text}
    Answer:"""

        answer = response_model.invoke(prompt)
        return answer
        # print("Context:", context[:300])
        # prompt = f"Use only the following context to answer:\n{context}\n\nUser question: {query_text}\nAnswer:"
        # print("Prompt:", prompt[:300])
        # answer = response_model.invoke(prompt)
        # print("LLM Answer:", answer)
        # return answer
    except Exception as e:
        print("ERROR OCCURRED IN pdf_qa_pipeline!")
        traceback.print_exc()
        return f"An error occurred: {str(e)}"

Using device: cpu


C:\Users\USER\AppData\Local\Temp\ipykernel_10692\4109286748.py:10: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(
C:\Users\USER\AppData\Local\Temp\ipykernel_10692\4109286748.py:14: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  response_model = Ollama(model="qwen3:4b")


In [20]:
# with open("/content/1602.04938v3.pdf", "rb") as pdf_file:
#   bytes_data = pdf_file.read()          # works
#   file_name  = pdf_file.name            # works  (e.g., '/content/1602.04938v3.pdf')


In [21]:
# with open("/content/1602.04938v3.pdf", "rb") as pdf_file:
#     pdf_qa_pipeline(pdf_file, "what is LIME ?")
# pdf_qa_pipeline(pdf_file, "what is LIME ?")

In [ ]:
demo = gr.Interface(
    fn=pdf_qa_pipeline,
    inputs=[
        gr.File(label="Upload PDF", file_types=[".pdf"]),
        gr.Textbox(label="Question", placeholder="Ask anything about the PDF..."),
    ],
    outputs=gr.Textbox(label="Answer"),
    title="PDF Q&A - All In One",
    description="Upload a PDF and ask a question—get an LLM answer grounded only in your document."
)

if __name__ == "__main__":
    demo.launch(mcp_server=True)

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.

🔨 MCP server (using SSE) running at: http://127.0.0.1:7860/gradio_api/mcp/sse


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


Starting pipeline...
DEBUG: pdf_file type: <class 'gradio.utils.NamedString'>
DEBUG: pdf_file dir: ['__add__', '__class__', '__contains__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getnewargs__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__mod__', '__module__', '__mul__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__rmod__', '__rmul__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'capitalize', 'casefold', 'center', 'count', 'encode', 'endswith', 'expandtabs', 'find', 'format', 'format_map', 'index', 'isalnum', 'isalpha', 'isascii', 'isdecimal', 'isdigit', 'isidentifier', 'islower', 'isnumeric', 'isprintable', 'isspace', 'istitle', 'isupper', 'join', 'ljust', 'lower', 'lstrip', 'maketrans', 'name', 'partition', 'removeprefix', 'removesuffix', 'replace', 'rfind', 'rindex', 'rjust', 'rpartition', 'rspl

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


Vectorstore built.
Retriever tool built.
Retrieved docs: [10].Forthetargetdomains,weselectedtheDOTAandDIOR in the 5-shot setting, the best mAPs are 32.45 (rank 128) on
datasets,whichwereconvertedtoCOCOformat1.Tosimulate DIOR and 22.91 (rank 32) on DOTA. In higher-shot settings,
a few-shot setting, we randomly selected k images per class the baseline remains competitive, but LoRA after fine-tuning
fortraining,wherekrepresentsthenumberofshots.Toensure closely matches its performance.
afaircomparisonandaccountforvariabilityintheselectionof LoRA after intermediate fine-tuning slightly improves
images,werepeatedeachexperiment5timesandreportedthe performance in low-shot settings, while the baseline remains
average results. Given that DOTA images often contain more strong in higher-shot configurations. The choice of rank in
than 100 objects, we set the maximum detection threshold to LoRA has a moderate impact on performance, with lower
300 in the pycocoapi evaluation toolkit [25]. ranks(e.g.,